In [ ]:
from pyspark.sql import functions as F

# ============================
# CONFIG
# ============================
CATALOG = "mlops_prod"
CONTROL_SCHEMA = "platform"
CUSTOMER_TABLE = f"{CATALOG}.{CONTROL_SCHEMA}.customers_master"

print(f"✅ Reading customers from: {CUSTOMER_TABLE}")

In [ ]:
# ============================
# 1. READ ACTIVE CUSTOMERS
# ============================
customers_df = (
    spark.table(CUSTOMER_TABLE)
         .filter(F.col("is_active") == True)
         .select("customer_id", "schema_name")
)

customers = customers_df.collect()
print(f"✅ Found {len(customers)} active customers")


In [ ]:

# ============================
# 2. BOOTSTRAP EACH CUSTOMER
# ============================
for row in customers:
    customer_id = row["customer_id"]
    schema_name = row["schema_name"]   # usually same as customer_id

    full_schema = f"{CATALOG}.{schema_name}"
    print(f"\n🚀 Bootstrapping customer: {customer_id}")
    print(f"📁 Schema: {full_schema}")

    # ----------------------------
    # ✅ 2.1 CREATE SCHEMA
    # ----------------------------
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {full_schema}")
    print(f"✅ Schema ready: {full_schema}")

    # ----------------------------
    # ✅ 2.2 CREATE BRONZE TABLE
    # ----------------------------
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {full_schema}.bronze_raw (
            source_file     STRING,
            raw_payload     STRING,
            ingest_ts       TIMESTAMP
        )
        USING DELTA
    """)
    print("✅ Bronze table created")

    # ----------------------------
    # ✅ 2.3 CREATE SILVER TABLE
    # ----------------------------
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {full_schema}.silver_clean (
            record_id       STRING,
            clean_payload   STRING,
            process_ts      TIMESTAMP
        )
        USING DELTA
    """)
    print("✅ Silver table created")

    # ----------------------------
    # ✅ 2.4 CREATE GOLD TABLE
    # ----------------------------
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {full_schema}.gold_features (
            feature_1   DOUBLE,
            feature_2   DOUBLE,
            label       INT,
            train_ts    TIMESTAMP
        )
        USING DELTA
    """)
    print("✅ Gold table created")

    # ----------------------------
    # ✅ 2.5 CREATE VOLUMES (FILES / EXPORTS / ARTIFACTS)
    # ----------------------------
    try:
        spark.sql(f"""
            CREATE VOLUME IF NOT EXISTS {full_schema}.artifacts
        """)
        spark.sql(f"""
            CREATE VOLUME IF NOT EXISTS {full_schema}.exports
        """)
        spark.sql(f"""
            CREATE VOLUME IF NOT EXISTS {full_schema}.reports
        """)
        print("✅ Volumes created (artifacts, exports, reports)")
    except Exception as e:
        print("⚠️ Volume creation skipped (may not be enabled on Trial):", e)

    # ----------------------------
    # ✅ 2.6 LOGICAL MODEL REGISTRY NAME (NO PHYSICAL CREATE NEEDED)
    # ----------------------------
    model_name = f"{customer_id}_baseline_model"
    print(f"✅ Model registry name reserved: {model_name}")

print("\n✅ ✅ CUSTOMER BOOTSTRAP INFRA COMPLETED SUCCESSFULLY ✅")